## 作業

In [1]:
import requests
res = requests.get('https://news.ltn.com.tw/list/breakingnews')
res

<Response [200]>

In [3]:
#res.text

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')

In [6]:
#soup

In [16]:
for li in soup.select('.list li'):
  #print(li)
  title = li.select_one('span.title').text
  dt = li.select_one('.time').text
  link = li.select_one('a.tit').get('href')
  print(title, dt, link)
  print('======================================')

剪接影片造假談話 CNN：拜登用假廣告攻擊川普 09:34 https://news.ltn.com.tw/news/world/breakingnews/3296734
9/19重要財經新聞一覽 09:32 https://ec.ltn.com.tw/article/breakingnews/3296759
米奇米妮萌助陣！這檔購物節超划算 3000現抵300特惠組最低46折 09:30 https://ent.ltn.com.tw/news/breakingnews/3296637
安倍參拜靖國神社 報告已卸下首相職務 09:29 https://news.ltn.com.tw/news/world/breakingnews/3296750
追思李登輝》蔡總統親頒褒揚令 追思文感念「把民主自由留給台灣」 09:28 https://news.ltn.com.tw/news/politics/breakingnews/3296678
追思李登輝》「三十九與九十八」 李坤儀撰文悼念祖父慈愛 09:27 https://news.ltn.com.tw/news/politics/breakingnews/3296736
NBA》詹皇僅拿16張第1名選票  湖人主帥仍力挺：他才是MVP 09:21 https://sports.ltn.com.tw/news/breakingnews/3296723
NBA》重提往事倡導精神健康 「愛神」自曝：好友救了我一命 09:20 https://sports.ltn.com.tw/news/breakingnews/3296707
意外捲入輕生事件 郭雪芙到處傻問「誰墜樓過」 09:20 https://ent.ltn.com.tw/news/breakingnews/3296654
MLB》洋基「園丁」發願再戰2021 美球迷無情狠酸：拜託不要 09:17 https://sports.ltn.com.tw/news/breakingnews/3296695
科技股下挫牽動美各大指數收黑 道瓊跌244點 09:16 https://ec.ltn.com.tw/article/breakingnews/3296681
追思李登輝》兩組民眾疑追思會場周邊抗議 警方帶至陳抗區 09:15 https://news.ltn.com.t

In [17]:
newsary = []
for li in soup.select('.list li'):
  title = li.select_one('span.title').text
  dt = li.select_one('.time').text
  link = li.select_one('a.tit').get('href')
  newsary.append({'title':title, 'dt':dt, 'link':link})
  

In [18]:
newsary[0]

{'dt': '09:34',
 'link': 'https://news.ltn.com.tw/news/world/breakingnews/3296734',
 'title': '剪接影片造假談話 CNN：拜登用假廣告攻擊川普'}

In [19]:
import pandas
newsdf = pandas.DataFrame(newsary)

In [21]:
newsdf.head()

,title,dt,link
0,剪接影片造假談話 CNN：拜登用假廣告攻擊川普,09:34,https://news.ltn.com.tw/news/world/breakingnew...
1,9/19重要財經新聞一覽,09:32,https://ec.ltn.com.tw/article/breakingnews/329...
2,米奇米妮萌助陣！這檔購物節超划算 3000現抵300特惠組最低46折,09:30,https://ent.ltn.com.tw/news/breakingnews/3296637
3,安倍參拜靖國神社 報告已卸下首相職務,09:29,https://news.ltn.com.tw/news/world/breakingnew...
4,追思李登輝》蔡總統親頒褒揚令 追思文感念「把民主自由留給台灣」,09:28,https://news.ltn.com.tw/news/politics/breaking...


In [22]:
newsdf.to_excel('ltnnews.xlsx')

## 完整自由時報爬蟲

In [25]:
import requests
res = requests.get('https://news.ltn.com.tw/ajax/breakingnews/all/2')
jd = res.json()

In [28]:
newsdf = pandas.DataFrame(jd['data'].values())

In [30]:
newsdf.head(3)

,no,title,bigphoto_flag,photo_S,photo_L,url,time,type_en,group,type_cn,local,summary,video,localUrl,tagUrl,style,tagText
0,3296731,追思李登輝》李坤儀手捧骨灰罈 移靈車隊由翠山莊出發,0,http://img.ltn.com.tw/Upload/news/250/2020/09/...,http://img.ltn.com.tw/Upload/news/600/2020/09/...,https://news.ltn.com.tw/news/politics/breaking...,09:13,politics,breakingnews,政治,臺北市,前總統李登輝追思告別禮拜今天在淡水真理大學大禮拜堂舉行，移靈車隊上午約8時45分從翠山莊出發...,0,list/breakingnews/Taipei,list/breakingnews/politics,None,政治
1,3296724,營區停車場猥褻女同袍：「妹妹，你可以幫我嗎？」,0,http://img.ltn.com.tw/Upload/news/250/2020/09/...,http://img.ltn.com.tw/Upload/news/600/2020/09/...,https://news.ltn.com.tw/news/society/breakingn...,09:10,society,breakingnews,社會,高雄市,高市江姓職業軍人藉故約女同袍小真（化名）到營區停車場，在自小客車內涉嫌強行親吻、揉胸，甚至抓...,0,list/breakingnews/Kaohsiung,list/breakingnews/society,None,社會
2,3296649,「雙野CP」默契十足 綾野剛狂嗑烏龍麵直到極限,0,http://img.ltn.com.tw/Upload/ent/page/400S/202...,http://img.ltn.com.tw/Upload/ent/page/800S/202...,https://ent.ltn.com.tw/news/breakingnews/3296649,09:10,entertainment,breakingnews,娛樂,False,星野源、綾野剛主演TBS台日劇《MIU404》日前播畢，全劇收視率創下每集均超過雙位數的亮眼...,0,,list/breakingnews/entertainment,None,娛樂


In [33]:
import requests
import time
newsdfs = []
for i in range(2,11):
  res = requests.get('https://news.ltn.com.tw/ajax/breakingnews/all/{}'.format(i))
  jd = res.json()
  newsdf = pandas.DataFrame(jd['data'].values())
  time.sleep(1)
  newsdfs.append(newsdf)

In [35]:
type(newsdfs)

list

In [37]:
type(newsdfs[0])

pandas.core.frame.DataFrame

In [39]:
res = pandas.concat(newsdfs)

In [43]:
import jieba
words = []
for title in res['title'].tolist():
  words = words + jieba.lcut(title)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.915 seconds.
Prefix dict has been built successfully.


In [46]:
from collections import Counter
words = [word for word in words if len(word) >=2]
c = Counter(words)
c.most_common(10)

[('中職', 12),
 ('追思', 7),
 ('12', 7),
 ('中國', 7),
 ('李登輝', 6),
 ('影音', 6),
 ('兄弟', 6),
 ('網球', 5),
 ('NBA', 5),
 ('回應', 5)]

## Concat v.s. DataFrame

In [48]:
# list of dictionaries
ary = [
 {'title':'abc', 'link':'http://abc.com'},
 {'title':'def', 'link':'http://def.com'}
]

import pandas
df = pandas.DataFrame(ary)
df

,title,link
0,abc,http://abc.com
1,def,http://def.com


In [51]:
ary1 = [
 {'title':'abc', 'link':'http://abc.com'},
 {'title':'def', 'link':'http://def.com'}
]

ary2 = [
 {'title':'abc1', 'link':'http://abc1.com'},
 {'title':'def1', 'link':'http://def1.com'}
]

In [52]:
df1 = pandas.DataFrame(ary1)
df1

,title,link
0,abc,http://abc.com
1,def,http://def.com


In [55]:

df2 = pandas.DataFrame(ary2)
df2

,title,link
0,abc1,http://abc1.com
1,def1,http://def1.com


In [56]:
# list of dataframes
pandas.concat([df1, df2])

,title,link
0,abc,http://abc.com
1,def,http://def.com
0,abc1,http://abc1.com
1,def1,http://def1.com


## Lists

In [57]:
a = [1,2,3,'hello', [4,5,6]]

In [58]:
a.append({'a':1})

In [59]:
a

[1, 2, 3, 'hello', [4, 5, 6], {'a': 1}]

In [60]:
a = [1,3,5,7,9]
b = [2,4,6,8,10]

In [62]:
for i in range(len(a)):
  #print(i)
  print(a[i] * b[i])

2
12
30
56
90


In [65]:
for i,j in zip(a,b):
  print(i*j)

2
12
30
56
90


In [66]:
a * b

TypeError: ignored

In [67]:
type(a)

list

In [68]:
import numpy as np
na = np.array(a)

In [69]:
type(na)

numpy.ndarray

In [71]:
a

[1, 3, 5, 7, 9]

In [70]:
na

array([1, 3, 5, 7, 9])

In [72]:
na = np.array(a)
nb = np.array(b)
na * nb

array([ 2, 12, 30, 56, 90])

In [75]:
import numpy as np
my_list = [1,2,3]
na_list = np.array(my_list)

In [77]:
na_list * 3

array([3, 6, 9])

In [78]:
np.sqrt(na_list)

array([1.        , 1.41421356, 1.73205081])

In [80]:
my_matrix =[[1,2,3],[4,5,6],[7,8,9]]
m = np.array(my_matrix)

In [81]:
-1 * m 

array([[-1, -2, -3],
       [-4, -5, -6],
       [-7, -8, -9]])

In [82]:
np.arange(0,10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [83]:
np.arange(0,11,2)

array([ 0,  2,  4,  6,  8, 10])

In [84]:
np.zeros(3)

array([0., 0., 0.])

In [85]:
np.zeros((5,5))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [86]:
np.ones((5,5))

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [87]:
np.random.randint(1,42,6)

array([26,  8, 20, 29,  4, 30])

In [88]:
arr = np.arange(25)

In [89]:
arr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [90]:
arr.shape

(25,)

In [91]:
arr.reshape(5,5)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [93]:
a = np.array([70,60,43,66,82,90,53,95,88])
a.max()

95

In [94]:
a.min()

43

In [95]:
a.argmax()

7

In [96]:
a.argmin()

2

In [100]:
x = np.array([20,30,10,50,40])
w = np.random.randn(5,7)
o1 = np.dot(x,w)
w1 = np.random.randn(7,4)
o2 = np.dot(o1,w1)

In [101]:
o2

array([  97.25084349,   47.7397188 , -100.92377224,  -41.63147988])

In [102]:
arr = np.arange(0,11)

In [104]:
arr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [105]:
arr[8]

8

In [106]:
arr[1:5]

array([1, 2, 3, 4])

In [107]:
arr = np.arange(0,11)

In [108]:
slice_of_arr = arr[0:6]

In [109]:
slice_of_arr

array([0, 1, 2, 3, 4, 5])

In [110]:
slice_of_arr[:]=99

In [111]:
arr

array([99, 99, 99, 99, 99, 99,  6,  7,  8,  9, 10])

In [112]:
arr_2d = np.array(([5,10,15],[20,25,30],[35,40,45]))

In [113]:
arr_2d

array([[ 5, 10, 15],
       [20, 25, 30],
       [35, 40, 45]])

In [115]:
arr_2d[1]

array([20, 25, 30])

In [116]:
arr_2d[1][0]

20

In [117]:
arr_2d[1,0]

20

In [118]:
arr_2d[2]

array([35, 40, 45])

In [119]:
#[列,欄]
arr_2d[2,:]

array([35, 40, 45])

In [121]:
arr = np.arange(1,11)

In [122]:
arr

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [124]:
arr > 4

array([False, False, False, False,  True,  True,  True,  True,  True,
        True])

In [125]:
bool_ary = arr > 4

In [127]:
arr[bool_ary]

array([ 5,  6,  7,  8,  9, 10])

In [128]:
arr = np.arange(0,10)

In [129]:
arr + arr

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [130]:
arr - arr

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [131]:
arr * arr

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [132]:
arr / arr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([nan,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [ ]:
# nan => not a number, (missing value)

In [135]:
# inf => infiinity
1/arr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


array([       inf, 1.        , 0.5       , 0.33333333, 0.25      ,
       0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111])

In [134]:
arr ** 3

array([  0,   1,   8,  27,  64, 125, 216, 343, 512, 729])

In [136]:
np.sqrt(arr)

array([0.        , 1.        , 1.41421356, 1.73205081, 2.        ,
       2.23606798, 2.44948974, 2.64575131, 2.82842712, 3.        ])

In [137]:
np.exp(arr)


array([1.00000000e+00, 2.71828183e+00, 7.38905610e+00, 2.00855369e+01,
       5.45981500e+01, 1.48413159e+02, 4.03428793e+02, 1.09663316e+03,
       2.98095799e+03, 8.10308393e+03])

In [138]:
np.max(arr)


9

In [139]:
np.sin(arr)


array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
       -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825,  0.41211849])

In [140]:
np.log(arr)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


array([      -inf, 0.        , 0.69314718, 1.09861229, 1.38629436,
       1.60943791, 1.79175947, 1.94591015, 2.07944154, 2.19722458])

### Practice
今天有三個學生, David, John, Mary, 她們考的兩次考試成績, 分別為 David: 90, 80,
John: 77, 65, Mary:88,95
請將這三人的考試成績表示成 2維的numpy array
(列:人, 欄: 考試次數)

今天老師給的加權分數為第一次考試為40%, 第二次考試為60%, 請利用該加權分數, 計算他們三人的最後總分?
